<a href="https://colab.research.google.com/github/shlok-mehndiratta/car_chatbot/blob/main/CAR_Chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

5th December 2025

In [1]:
import os
import json
import re
import textwrap
import requests

# Base project folder
BASE_DIR = "/content/CarleaseBot"
CONTRACTS_DIR = os.path.join(BASE_DIR, "contracts")
OUTPUT_DIR = os.path.join(BASE_DIR, "outputs")

os.makedirs(CONTRACTS_DIR, exist_ok=True)
os.makedirs(OUTPUT_DIR, exist_ok=True)

print("Project folders created:")
print("BASE_DIR     :", BASE_DIR)
print("CONTRACTS_DIR:", CONTRACTS_DIR)
print("OUTPUT_DIR   :", OUTPUT_DIR)

Project folders created:
BASE_DIR     : /content/CarleaseBot
CONTRACTS_DIR: /content/CarleaseBot/contracts
OUTPUT_DIR   : /content/CarleaseBot/outputs


In [2]:
!pip install pdfplumber

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.7/67.7 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 39.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 28.8 MB/s eta 0:00:00


In [3]:
from google.colab import files
import shutil

print("Please upload one or more contract files (PDF preferred).")
uploaded = files.upload()   # choose PDFs from your system

for filename in uploaded.keys():
    src = os.path.join("/content", filename)
    dst = os.path.join(CONTRACTS_DIR, filename)
    shutil.move(src, dst)
    print("Saved:", dst)

print("\nFiles in contracts folder:")
!ls -l "$CONTRACTS_DIR"

Please upload one or more contract files (PDF preferred).


Saving Car Lease Agreement Template (New).pdf to Car Lease Agreement Template (New).pdf
Saved: /content/CarleaseBot/contracts/Car Lease Agreement Template (New).pdf

Files in contracts folder:
total 184
-rw-r--r-- 1 root root 185117 Dec  7 15:57 'Car Lease Agreement Template (New).pdf'


In [4]:
import pdfplumber

def extract_text_from_pdf(pdf_path: str) -> str:
    """Extracts text from all pages of a PDF file."""
    text_pages = []
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            page_text = page.extract_text() or ""
            text_pages.append(page_text)
    full_text = "\n".join(text_pages)
    return full_text

# Preview first uploaded contract
def preview_first_contract():
    files_in_dir = sorted(os.listdir(CONTRACTS_DIR))
    if not files_in_dir:
        print("No contracts found in:", CONTRACTS_DIR)
        return None, None

    first_file = files_in_dir[0]
    pdf_path = os.path.join(CONTRACTS_DIR, first_file)

    text = extract_text_from_pdf(pdf_path)
    print("Showing first 1000 characters of:", first_file)
    print("-" * 80)
    print(text[:1000])
    print("-" * 80)
    return pdf_path, text

first_pdf_path, first_pdf_text = preview_first_contract()

Showing first 1000 characters of: Car Lease Agreement Template (New).pdf
--------------------------------------------------------------------------------
Car Lease
Agreement
Template
Prepared for Prepared by
[Recipient Name] [Sender Name]
[Company Name] [Company Name]
Car Lease Agreement Template
1 . Introduction
This Car Lease Agreement (the "Agreement") is entered into on this [__ day of _____,
20] by and between [Lessor's Full Name], hereinafter referred to as the "Lessor," and
[Lessee's Full Name], hereinafter referred to as the "Lessee."
This Agreement sets forth the terms and conditions upon which the Lessor agrees to
lease the vehicle described herein to the Lessee, and the Lessee agrees to lease the
said vehicle from the Lessor. The parties hereto mutually agree to abide by all the
terms and conditions stipulated within this Agreement.
2. Description of Vehicle
❖ Vehicle Make: [e.g., Toyota, Ford, etc.]
❖ Vehicle Model: [e.g., Camry, F-150, etc.]
❖ Vehicle Year: [Year of Manufa

In [11]:
import re
from typing import Dict, Any, Optional

def find_percentage(text: str) -> Optional[float]:
    """Find first percentage like '7.5%' in text and return as float."""
    match = re.search(r'(\d+(\.\d+)?)\s*%', text)
    if match:
        return float(match.group(1))
    return None

def extract_sla_fields(text: str) -> Dict[str, Any]:
    """
    Extracts key SLA fields using regex + heuristics.
    Works well for standard lease/loan documents.
    """
    lower = text.lower()

    # APR or interest rate
    apr = None
    for pat in [
        r"apr[^0-9]{0,10}(\d+(\.\d+)?)\s*%",
        r"interest\s+rate[^0-9]{0,10}(\d+(\.\d+)?)\s*%"
    ]:
        m = re.search(pat, lower, re.IGNORECASE)
        if m:
            apr = float(m.group(1))
            break
    if apr is None:
        apr = find_percentage(text)

    # Lease term (months)
    term_months = None
    m = re.search(r"(\d+)\s*(months?|mos\.)", lower)
    if m:
        term_months = int(m.group(1))

    # Monthly payment ($)
    monthly_payment = None
    m = re.search(r"\$?\s*([0-9,]+\.\d{2})\s*(per\s+month|monthly\s+payment)", lower)
    if m:
        monthly_payment = float(m.group(1).replace(",", ""))

    # Down payment
    down_payment = None
    m = re.search(r"(down\s+payment|amount\s+due\s+at\s+signing)[^0-9$]{0,20}\$?\s*([0-9,]+\.\d{2})", lower)
    if m:
        down_payment = float(m.group(2).replace(",", ""))

    # Mileage
    mileage_allowance = None
    m = re.search(r"(\d{5})\s*(miles\s*per\s*year|mi/yr)", lower)
    if m:
        mileage_allowance = int(m.group(1))

    # Excess mileage fee
    excess_fee = None
    m = re.search(r"(\d+(\.\d+)?)\s*cents?\s*(per\s*mile|/mi)", lower)
    if m:
        excess_fee = float(m.group(1))

    # Detect red flags
    red_flags = []
    if apr and apr > 15:
        red_flags.append(f"High APR: {apr}%")
    if term_months and term_months > 72:
        red_flags.append(f"Very long term ({term_months} months)")
    if excess_fee and excess_fee > 30:
        red_flags.append(f"High excess mileage fee: {excess_fee} cents/mile")

    return {
        "apr_percent": apr,
        "term_months": term_months,
        "monthly_payment": monthly_payment,
        "down_payment": down_payment,
        "mileage_allowance_per_year": mileage_allowance,
        "excess_mileage_fee_cents_per_mile": excess_fee,
        "red_flags": red_flags,
    }

In [6]:
# RUN ON YOUR FIRST CONTRACT
if first_pdf_text:
    sla = extract_sla_fields(first_pdf_text)
    print("📌 SLA summary:\n", sla)
else:
    print("No contract text loaded yet.")

def compute_fairness_score(sla: dict) -> dict:
    """
    Simple lease fairness score (0–100):
    - Deducts points for high APR, long term, and high monthly payments.
    - Uses red flag deductions as well.
    """
    score = 100
    reasons = []

    apr = sla.get("apr_percent")
    term = sla.get("term_months")
    monthly_payment = sla.get("monthly_payment")

    # APR logic
    if apr:
        if apr > 18:
            score -= 35; reasons.append("APR above 18%")
        elif apr > 12:
            score -= 20; reasons.append("APR between 12–18%")
        elif apr > 8:
            score -= 10; reasons.append("APR between 8–12%")
    else:
        reasons.append("APR missing or unclear")

    # Term logic
    if term:
        if term > 72:
            score -= 25; reasons.append("Term longer than 72 months")
        elif term > 60:
            score -= 15; reasons.append("Term longer than 60 months")

    # Payment logic
    if monthly_payment and monthly_payment > 600:
        score -= 15; reasons.append("Monthly payment above $600")

    # Red flags
    for flag in sla.get("red_flags", []):
        score -= 5
        reasons.append("Contract red flag: " + flag)

    score = max(0, min(100, score))

    return {
        "fairness_score": score,
        "reasons": reasons
    }


📌 SLA summary:
 {'apr_percent': 50.0, 'term_months': 12, 'monthly_payment': None, 'down_payment': None, 'mileage_allowance_per_year': None, 'excess_mileage_fee_cents_per_mile': None, 'red_flags': ['High APR: 50.0%']}


In [7]:
# RUN FAIRNESS EVALUATION
fairness = compute_fairness_score(sla)
print("📊 Fairness Score Results:\n", fairness)

import textwrap

def generate_negotiation_tips(sla: dict, fairness: dict) -> str:
    tips = []
    apr = sla.get("apr_percent")
    term = sla.get("term_months")
    monthly_payment = sla.get("monthly_payment")
    mileage = sla.get("mileage_allowance_per_year")
    excess_fee = sla.get("excess_mileage_fee_cents_per_mile")

    # APR negotiation
    if apr and apr > 12:
        tips.append(
            "APR appears high. Ask dealer to match your local bank or reduce APR by 1%–2%."
        )

    # Term negotiation
    if term and term > 60:
        tips.append(
            "Lease/loan term is long. Negotiate shorter terms (48–60 months) to avoid excessive interest."
        )

    # Mileage negotiation
    if mileage and mileage < 12000:
        tips.append(
            "Annual mileage allowance looks low. Increase mileage to avoid excess penalty."
        )

    # Excess mileage fee
    if excess_fee and excess_fee > 25:
        tips.append(
            "Excess mileage fee is high. Request a reduced per-mile fee or higher mileage package."
        )

    # Monthly payment
    if monthly_payment and monthly_payment > 600:
        tips.append(
            "Monthly payment is high. Ask to reduce APR, term or increase down payment."
        )

    # Generic suggestion
    if not tips:
        tips.append(
            "Terms appear reasonable, but request a written breakdown of all fees to validate hidden charges."
        )

    # Always recommended
    tips.append(
        "Ask for detailed fee disclosure (doc fee, acquisition fee, add-ons) and remove non-essential extras."
    )

    wrapped = "\n\n".join(textwrap.fill(t, width=90) for t in tips)
    return wrapped



📊 Fairness Score Results:
 {'fairness_score': 60, 'reasons': ['APR above 18%', 'Contract red flag: High APR: 50.0%']}


In [8]:
# RUN NEGOTIATION ADVISOR
tips_output = generate_negotiation_tips(sla, fairness)
print("💬 Negotiation Recommendations:\n")
print(tips_output)

import json

def analyze_contract(filename: str):
    """
    Full pipeline for a single contract:
    1. Extract text
    2. Extract SLA fields
    3. Compute fairness score
    4. Generate negotiation tips
    5. Save analysis to JSON
    """

    pdf_path = os.path.join(CONTRACTS_DIR, filename)
    if not os.path.exists(pdf_path):
        raise FileNotFoundError(f"Contract not found: {pdf_path}")

    # Extract text
    text = extract_text_from_pdf(pdf_path)

    # SLA extraction
    sla = extract_sla_fields(text)

    # Fairness score
    fairness = compute_fairness_score(sla)

    # Negotiation advice
    tips = generate_negotiation_tips(sla, fairness)

    # Output container
    result = {
        "file": filename,
        "sla": sla,
        "fairness": fairness,
        "negotiation_tips": tips
    }

    # Save JSON
    out_path = os.path.join(OUTPUT_DIR, filename + ".analysis.json")
    with open(out_path, "w") as f:
        json.dump(result, f, indent=2)

    print("📁 Analysis saved to:", out_path)

    print("\n=== SLA FIELDS ===")
    print(json.dumps(sla, indent=2))

    print("\n=== FAIRNESS SCORE ===")
    print(json.dumps(fairness, indent=2))

    print("\n=== NEGOTIATION RECOMMENDATIONS ===")
    print(tips)

    return result

💬 Negotiation Recommendations:

APR appears high. Ask dealer to match your local bank or reduce APR by 1%–2%.

Ask for detailed fee disclosure (doc fee, acquisition fee, add-ons) and remove non-
essential extras.


In [9]:

# Automatically run on the first uploaded contract
files_in_dir = sorted(os.listdir(CONTRACTS_DIR))
if files_in_dir:
    final_report = analyze_contract(files_in_dir[0])
else:
    print("No contract found in contracts folder.")

def analyze_all_contracts():
    files_in_dir = sorted(os.listdir(CONTRACTS_DIR))
    if not files_in_dir:
        print("No contract files found in:", CONTRACTS_DIR)
        return

    print(f"Found {len(files_in_dir)} contract(s). Running batch analysis...\n")

    for filename in files_in_dir:
        print("\n==========================================")
        print(f"Analyzing: {filename}")
        print("==========================================")
        analyze_contract(filename)

    print("\n✅ Batch analysis completed!")

# RUN THIS
analyze_all_contracts()

📁 Analysis saved to: /content/CarleaseBot/outputs/Car Lease Agreement Template (New).pdf.analysis.json

=== SLA FIELDS ===
{
  "apr_percent": 50.0,
  "term_months": 12,
  "monthly_payment": null,
  "down_payment": null,
  "mileage_allowance_per_year": null,
  "excess_mileage_fee_cents_per_mile": null,
  "red_flags": [
    "High APR: 50.0%"
  ]
}

=== FAIRNESS SCORE ===
{
  "fairness_score": 60,
  "reasons": [
    "APR above 18%",
    "Contract red flag: High APR: 50.0%"
  ]
}

=== NEGOTIATION RECOMMENDATIONS ===
APR appears high. Ask dealer to match your local bank or reduce APR by 1%–2%.

Ask for detailed fee disclosure (doc fee, acquisition fee, add-ons) and remove non-
essential extras.
Found 1 contract(s). Running batch analysis...


Analyzing: Car Lease Agreement Template (New).pdf
📁 Analysis saved to: /content/CarleaseBot/outputs/Car Lease Agreement Template (New).pdf.analysis.json

=== SLA FIELDS ===
{
  "apr_percent": 50.0,
  "term_months": 12,
  "monthly_payment": null,
  "dow

In [10]:
!pip install reportlab

from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer
from reportlab.lib.pagesizes import LETTER
from reportlab.lib.styles import getSampleStyleSheet

from datetime import datetime
import json
import os

def generate_contract_report(result: dict, filename: str):
    report_path = os.path.join(OUTPUT_DIR, filename + "_REPORT.pdf")
    styles = getSampleStyleSheet()
    doc = SimpleDocTemplate(report_path, pagesize=LETTER)
    story = []

    story.append(Paragraph("CAR LEASE CONTRACT ANALYSIS REPORT", styles['Title']))
    story.append(Spacer(1, 12))

    story.append(Paragraph(f"Contract File: {result['file']}", styles['Normal']))
    story.append(Paragraph(f"Generated On: {datetime.now()}", styles['Normal']))
    story.append(Spacer(1, 12))

    story.append(Paragraph("SLA SUMMARY", styles['Heading2']))
    for k, v in result['sla'].items():
        story.append(Paragraph(f"{k}: {v}", styles['Normal']))
    story.append(Spacer(1, 12))

    story.append(Paragraph("FAIRNESS SCORE", styles['Heading2']))
    story.append(Paragraph(f"Score: {result['fairness']['fairness_score']}", styles['Normal']))
    story.append(Paragraph("Reasons:", styles['Normal']))
    for r in result['fairness']['reasons']:
        story.append(Paragraph(f"- {r}", styles['Normal']))
    story.append(Spacer(1, 12))

    story.append(Paragraph("NEGOTIATION RECOMMENDATIONS", styles['Heading2']))
    for line in result['negotiation_tips'].split("\n"):
        story.append(Paragraph(line, styles['Normal']))
    story.append(Spacer(1, 12))

    doc.build(story)
    return report_path

all_reports = []

for filename in os.listdir(OUTPUT_DIR):
    if filename.endswith(".analysis.json"):
        json_path = os.path.join(OUTPUT_DIR, filename)
        with open(json_path, "r") as f:
            data = json.load(f)
        pdf_path = generate_contract_report(data, data['file'])
        all_reports.append(pdf_path)
        print("📄 PDF Created:", pdf_path)

print("\n🎉 ALL REPORTS GENERATED!")

from google.colab import files
files.download(all_reports[0])

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 21.5 MB/s eta 0:00:00
📄 PDF Created: /content/CarleaseBot/outputs/Car Lease Agreement Template (New).pdf_REPORT.pdf

🎉 ALL REPORTS GENERATED!


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>